# Testing: What, Why and How To

## The Many Types of [Software] Tests

There are many types of tests broadly divided into two categories:

* **functional testing** ⬜ 📜 testing the functionality
* **non-functional testing** 💣 🔨 testing the system as a whole without regard for intended functionality

Another dimension you can categorize tests by is whether they are:
* **automated** 🤖
* or **manual** 🧑

The specific categories of tests you will most likely see out in the wild include:

* **unit** testing (atomic functionality)
* **integration** testing (interrelated functionality)
* **user acceptance** testing (does it fulfill the user contract)
* **compliance** testing (does it comply with prescribed regulations)
* **end-to-end** testing (full user/system workflows)
* **usability** testing (human friendly)
* **accessibility** testing (disability friendly)
* **load** testing (regular traffic)
* **stress** testing (worst case traffic)
* **penetration** testing (hackers)
* **fuzz** testing (random inputs)
* **formatting** testing (does the code conform to defined formats)

#### We will be focusing on functional, automated, **unit tests**.

## Why should we take the time to write automated tests?

Testing is a fundamental practice in software development. It allows us to:

* Tests provide **documentation** 📜 of how our code is supposed to work for collaborators and for our future selves

* Tests help **engineers onboard into established codebases** without anxiety 😬 🙈

* Regular testing helps maintain the reliability and stability of our codebase so that we can **confidently expand** our projects without time wasted tracking down regressions

* Ensure code quality by **catching bugs 🐛 early** and keeping them **out of production** 🚨

 There are **many languages** out there. We have to choose one, so we'll use Python here. The **high level concepts and features of our testing framework** are **fairly universal**.

## Test-Driven Development

Test-Driven Development (TDD) is an iterative software development approach that encourages writing tests before writing the actual code.

### Fail Fast 💣 🚀

Writing the test first makes you stop to **carefully consider** what a function really needs to do. You're writing a specification, otherwise referred to as a requirements document, before you write your "real" code.

You will sometimes see the TDD cycle referred to as:

### 🔴🟢🔄 Red Green Refactor

1. 🔴 **Write** a test that defines the expected behavior and outcomes of the function
2. 🔴 **Run** the test and see that it fails because there's no functionality defined yet
3. 🟢 **Write** the minimum code to pass the test
4. 🟢 **Run** the test... see that it succeeds (if it fails... keep at it!)
5. 🔄 **Check** over your function and your test, make any **tweaks** to make the code better or more descriptive/self-documenting/faster

**Cycle** between tweaking the function and running the test until the test passes and you are happy with the functionality and the test code.

Working in this way forces you to be **intentional** about the code you write and to really think through what the inputs and outputs should be. It also prevents the inevitable headache of debugging an **"over-engineered" monolith** of code... which is extremely demoralizing.

The resulting code will be more **modular** and decoupled, making it more **reusable**. You'll find your code being used by other engineers because your test-covered, self-documenting functions are consistently reliable.

## The pytest Testing Framework

`unittest` is baked in to Python, but we'll be using [pytest](https://realpython.com/pytest-python-testing/)  
- less setup
- simple syntax
- large ecosystem of [plugins](https://pytest.org/en/stable/reference/plugin_list.html)

- auto-finds tests, just as long as you make sure your test files and functions start with `test_`!

### First Unit Test

We're going to write some utility functions for working with grades. First lets set up our environment.

In [ ]:
%%bash
# set up code directory
mkdir -p gradebook

# colab will complain later without this
#apt install python3.10-venv

In [ ]:
# set up python
!python3 -m venv venv
!source venv/bin/activate
!python3 -m pip install pytest

We have to remember a few things as we start to write unit tests.

* Unit tests are designed to **test individual bits of logic** ("units"), self-contained, **no dependencies**
* Each test should focus on a single behavior or outcome, i.e. **don't try to cover every possible scenario or outcome in one test**

You should approach building unit tests keeping in mind 3 steps:


#### 1. Arrange
set up the conditions for your test

#### 2. Act
invoke the unit of code being tested

#### 3. Assert
check that the result is what was expected

In [ ]:
!mkdir test

In [ ]:
%%writefile test/test_averaging.py
import pytest
from gradebook.grade_utils import calculate_average

def test_averaging():
    grades = [90, 80, 70] # arrange
    average = calculate_average(grades) # act
    assert average == 80 # assert

Now lets run `pytest`. Remember that we're **anticipating failure.**

In [ ]:
!python3 -m pytest

**That looks about right.**

Lets add a basic implementation.

In [ ]:
!touch gradebook/__init__.py

In [ ]:
%%writefile gradebook/grade_utils.py

def calculate_average(grades):
    return sum(grades) / len(grades)


Re-run our test...

In [ ]:
!python3 -m pytest

I do 💚 love 💚 seeing that <span style='color:green'>green</span>.

If you want to see which specific tests were executed, you can pass the `--verbose` or `-v` parameter to pytest.

In [ ]:
!python3 -m pytest -v

Test names should always be **descriptive**, so lets give it a better name for those who come after us, and trim it down to be more Python-y now that you get the **arrange-act-assert** point.

In [ ]:
%%writefile test/test_averaging.py
import pytest
from gradebook.grade_utils import calculate_average

def test_that_average_grade_returns_average_of_grades_provided():
    assert calculate_average([90, 80, 70]) == 80


In [ ]:
!python3 -m pytest -v

## Fixtures

Technically, we should not be hardcoding any values. We should use fixtures instead.

This helps by **documenting** the expected values for anyone who looks at this code and also facilitates their **adding more tests** by giving them building blocks to start with.

In [ ]:
%%writefile test/test_averaging.py
import pytest
from gradebook.grade_utils import calculate_average

@pytest.fixture
def some_grades():
    return [90, 80, 70]

def test_that_average_grade_returns_average_of_grades_provided(some_grades):
    assert calculate_average(some_grades) == 80

In [ ]:
!python3 -m pytest

**Maybe** we should discuss that `pytest` **abbreviated output**.

- A green dot `.` means that the test passed
- An `F` means that the test has failed
- An `E` means that the test raised an unexpected exception

Lets step back and consider whether this test covers every situation we may want to test for. 🤔

What if the list is **empty**?

In [ ]:
%%writefile test/test_averaging_some_more.py
import pytest
from gradebook.grade_utils import calculate_average

@pytest.fixture
def no_grades():
    return []

def test_that_average_of_no_grades_still_works(no_grades):
    assert calculate_average(no_grades) is None

In [ ]:
!python -m pytest -v

**Okay, now, lets **refactor** our function to handle the empty list.**

In [ ]:
%%writefile gradebook/grade_utils.py

def calculate_average(grades):
    if not grades:
      return None
    return sum(grades) / len(grades)

I maintain that the average of an empty list is nothing, sending a clear message that the list of grades was empty and not a set of zeros, which could be a valid input.

Gathering requirements is a topic for another day, but just wanted to point out how **writing tests first helps to shed light on ambiguity** so you can have a chat with the user requesting the change or the "product owner" in a larger org.

In [ ]:
!python -m pytest

And, you know, erring on the side of caution isn't a bad thing. It doesn't hurt to introduce tests just to make sure things work how you expect, e.g. if the input is not an empty list but nothing:

In [ ]:
%%writefile test/test_averaging_yet_again.py
import pytest
from gradebook.grade_utils import calculate_average

def test_that_average_of_nothing_still_does_what_is_expected():
    assert calculate_average(None) is None

In [ ]:
!python -m pytest -v

## Markers

In a **large codebase**, I have seen **automated tests take upwards of 30 minutes** to run because there are **so many**, each atomic, each "arranging" its setup. It may be beneficial to start using [markers](https://pytest-with-eric.com/pytest-best-practices/pytest-markers/) straight out of the gate so you can run only the tests in the scope you are modifying.

This will also come in handy if you are deploying to multiple platforms and your test is only relevant to **specific architectures** or **specific versions** of dependencies. You can then configure CI/CD to run only a subset of tests depending which version you are building.

**Markers** allow you to group your unit tests and run them by marker.

Lets apply markers to 2 of the 3 tests and see what happens.

In [ ]:
%%writefile test/test_averaging_yet_again.py
import pytest
from gradebook.grade_utils import calculate_average

@pytest.mark.average
def test_that_average_of_nothing_still_does_what_is_expected():
    assert calculate_average(None) is None

In [ ]:
%%writefile test/test_averaging_some_more.py
import pytest
from gradebook.grade_utils import calculate_average

@pytest.fixture
def no_grades():
    return []

@pytest.mark.average
def test_that_average_of_no_grades_still_works(no_grades):
    assert calculate_average(no_grades) is None

All three of our tests will still run with no parameters passed.

In [ ]:
!python -m pytest

But if we pass the marker parameter with the marker name "average"...

In [ ]:
!python -m pytest -m average

It's just a warning, but we should really register our markers, if only to keep a record of what markers are available when other engineers run our tests and want to find out what markers there are to choose from:

In [ ]:
!python -m pytest --markers

We can add our custom marker to a dedicated `pytest.ini` file or a `pyproject.toml` file. I would recommend adding to `pyproject.toml` if there is one. It's always good to go best practice instead of ignoring these warnings so let's configure a `pyproject.toml` so we stop seeing this warning in subsequent runs.

In [ ]:
%%writefile pyproject.toml
[tool.pytest.ini_options]
markers = [
    "average: used to mark all tests associated with averaging grades"
]

In [ ]:
!python -m pytest --markers

**Neat!**

There are many other configuration settings for `pytest`. Read more about them in the [official docs](https://docs.pytest.org/en/7.1.x/reference/reference.html#ini-options-ref).

Now lets see if the warning went away.

In [ ]:
!python -m pytest -m average

Love it! You can also **exclude** a marker instead of including it by quoting the marker and preceding with a `not`, e.g.:

In [ ]:
!python -m pytest -m "not average"

Lets go back and add that `average` marker to the 3rd test to be consistent.

In [ ]:
%%writefile test/test_averaging.py
import pytest
from gradebook.grade_utils import calculate_average

@pytest.fixture
def some_grades():
    return [90, 80, 70]

@pytest.mark.average
def test_that_average_grade_returns_average_of_grades_provided(some_grades):
    assert calculate_average(some_grades) == 80

Test that it worked. We should have no tests that are not marked average.

In [ ]:
!python -m pytest -m "not average"

Lets add a few more tests so we can see how fixtures can be shared. Here I will borrow the `some_grades` fixture from earlier in our new tests.

In [ ]:
%%writefile test/test_highest_lowest.py
import pytest
from gradebook.grade_utils import find_highest_grade, find_lowest_grade

@pytest.mark.hilo
@pytest.mark.highest
def test_find_highest_grade(some_grades):
    assert find_highest_grade(some_grades) == 90

@pytest.mark.hilo
@pytest.mark.lowest
def test_find_lowest_grade(some_grades):
    assert find_lowest_grade(some_grades) == 70

Notice that we are applying **multiple markers** to our new tests.

I will go ahead and create the logic as well.

In [ ]:
%%writefile gradebook/grade_utils.py

def calculate_average(grades):
    """Return the average grade"""
    if not grades:
      return None
    return sum(grades) / len(grades)


def find_highest_grade(grades):
    """Return the highest grade"""
    if not grades:
        return None
    return max(grades)


def find_lowest_grade(grades):
    """Return the lowest grade"""
    if not grades:
        return None
    return min(grades)


In [ ]:
!python -m pytest

Okay, so we have a few **new markers** that we will register, but the only **error** is that our fixture is `not found`. Lets address that fixture first. We can share fixtures with all of our tests. Lets move `some_grades` to a `conftest.py` (`pytest` does expect the shared fixtures to live in a file of this specific name) so that we can have a place to add fixtures as our codebase grows.

In [ ]:
%%writefile test/conftest.py
import pytest

@pytest.fixture(scope="function")
def some_grades():
    return [90, 80, 70]

When fixtures are defined in `conftest.py`, they can have a **scope** applied. The scope determines when the fixture will be invoked. In this case, this fixture will be invoked for each function in which it is used. This is actually the **default**.

There are several options because some fixtures are complex. You can instantiate a class or a service as a fixture, and you will not want **expensive operations** to be performed more often than necessary.

The other options are:

- class
- module
- session

[Real world example](https://github.com/getsentry/sentry/blob/a471c8ad53dcd956f6b886c06ee1555697966002/src/sentry/testutils/pytest/kafka.py#L14.) of why you might want to scope fixtures.

Lets **remove** the fixture from where it had originally been declared.

In [ ]:
%%writefile test/test_averaging.py
import pytest
from gradebook.grade_utils import calculate_average

@pytest.mark.average
def test_that_average_grade_returns_average_of_grades_provided(some_grades):
    assert calculate_average(some_grades) == 80

Lets also **register our new markers**.

In [ ]:
%%writefile pyproject.toml
[tool.pytest.ini_options]
markers = [
    "average: used to mark all tests associated with averaging grades",
    "hilo: used to mark tests associated with identifying the extreme grades",
    "highest: used to mark tests associated with identfying the highest grade",
    "lowest: used to mark tests associated with identifying the lowest grade"
]

After applying **multiple markers** to the same tests, we can now refer to any of the markers associated with a test to run that test.

In [ ]:
!python -m pytest -m hilo -v

Both of the tests in the hilo file ran!

And to demonstrate the point that we can use **any of the markers** applied to a test to run that test...

In [ ]:
!python -m pytest -m highest -v

Lots of marker settings available, including **conditional skipping** and **intentional failure**, which can be used as a placeholder to address an issue, and **timeout** markers that will cause the test to error if it exceeds the set time.

More examples with explanations can be found on this handy blog: https://pytest-with-eric.com/pytest-best-practices/pytest-markers/


Markers can also be used to [skip tests based on conditions](https://github.com/urllib3/urllib3/blob/da410581b6b3df73da976b5ce5eb20a4bd030437/dummyserver/testcase.py#L314).

Before we add more tests, some additional considerations...

* Tests should not be **interdependent**.
* Tests need to be able to run **in any order**.
* Unit tests should have a low **execution time** because there will be many of them.
* Tests need to be run **frequently**, preferably **automatically** as a pre-commit hook or, much better, as part of a CI workflow.

In [ ]:
!python -m pytest --durations=0 -vv

Lets add some more tests and some more functions and then rerun our new test suite.

In [ ]:
%%writefile test/test_integrated_functionality.py
import pytest
from gradebook.grade_utils import calculate_average, determine_letter_grade

def test_letter_grade_average(some_grades):
    # calculate_average
    average = calculate_average(some_grades)

    # determine_letter_grade for the average
    average_letter_grade = determine_letter_grade(average)
    assert average_letter_grade == "B"

**That's technically a super lightweight **integration** test because it checks that two different parts of our codebase work together.**

Let's add the code to return a letter grade for a numeric grade.

In [ ]:
%%bash
cat << _EOF >> gradebook/grade_utils.py


def determine_letter_grade(grade):
    """Return the letter grade for a numeric grade."""
    if grade >= 90:
        return "A"
    elif grade >= 80:
        return "B"
    elif grade >= 70:
        return "C"
    elif grade >= 60:
        return "D"
    else:
        return "F"
_EOF

In [ ]:
!python -m pytest

## Parametrization

Now that we've added the letter grade function, it would be a great time to bring up **parametrization**. Parametrization comes in handy if you find yourself writing a lot of tests that look very similar, or if you have to test for a long list of scenarios that could be easily expressed as parameters.

In [ ]:
%%writefile test/test_parametrized_letter_grades.py
import pytest

from gradebook.grade_utils import determine_letter_grade


grade_ranges = {
    "A": range(90, 101),
    "B": range(80, 90),
    "C": range(70, 80),
    "D": range(60, 70),
    "F": range(0, 60),
}

@pytest.mark.parametrize("letter,number",
                         [(letter, number) for letter, numbers in grade_ranges.items() for number in numbers])

def test_is_letter_grade(letter, number):
    assert determine_letter_grade(number) == letter

***Brace yourself...***

In [ ]:
!python -m pytest

In [ ]:
!python -m pytest -v

Now that we've looked at a silly example, a [real example of parametrization](https://github.com/tiangolo/fastapi/blob/a9819dfd8da39a754837cc134df4aca6c0a9a3f6/tests/test_param_include_in_schema.py#L168).

## Mocking Complex Dependencies

The last feature that will be essential as you begin to work with established codebases is **mocking**. [`pytest-mock`](https://pytest-mock.readthedocs.io/en/latest/usage.html) is a `pytest` plugin that provides mocking functionality. You should not connect to an actual database or your 3rd party API during unit testing. You should "mock" these dependencies instead.

### Why mock?
Because unit tests should be isolated, have **no dependencies**, and be **fast**, avoiding latency like that found in networking and database or disk operations

You can mock pretty much anything:
- HTTP requests
- Database query results
- File system manipulation
- Built-in functions and constants
- Complex classes we do not want to have to initialize
- 3rd party libraries

In [ ]:
!python -m pip install pytest-mock

Now we will create a simple function to write the grades list to a file.

In [ ]:
%%writefile gradebook/save_grades.py
def write_to_file(grades) -> None:
    """
    Function to write our grades to a file
    :param grades: grades list
    :return: None
    """
    with open(f"grades.txt", "w") as f:
        f.write(str(grades))


This function opens a file and writes the grades. Super simple. We don't want to actually open a file or actually write to a file, so we will mock the opening and writing. We will intercept these calls.

In [ ]:
%%writefile test/test_write_to_file.py
import pytest

from gradebook.save_grades import write_to_file

# first we pass the mocker in
def test_write_grades_to_file(mocker):
    """
    Function to test writing grades to a file
    """

    # mock the 'open' function call to return a file object
    # using a builtin from unittest
    mock_file = mocker.mock_open()
    mocker.patch("builtins.open", mock_file)

    # now we can call our function that writes to a file
    write_to_file([50,75,100])

    # assert that the 'open' function was called with the expected arguments
    mock_file.assert_called_once_with("grades.txt", "w")

    # assert that the file was written to with the expected text
    mock_file().write.assert_called_once_with(str([50,75,100]))

In [ ]:
!python -m pytest

Beyond mocks there are also **spies**, but mocks are plenty on their own. Read all about mocking features in the [docs](https://pytest-mock.readthedocs.io/en/latest/usage.html). Mocks replace functionality with hardcoded values and spies replace only portions of real classes/modules which can be useful when you want to make sure that a deeper method in a 3rd party library or legacy codebase was invoked.

#### Here's [a real mock example](https://github.com/microsoft/timewarp/blob/44dca8474cb6182458830677763261cffccfaac4/utilities/fixtures.py#L80) from Microsoft!

#### And a [real spy example](https://github.com/slackapi/python-slack-events-api/blob/2884d7d21fea634d1e5e7926409ed87f6fcc14cf/tests/test_server.py#L179) from Slack.

## Code Coverage

**Aim** for **high** code coverage, but don't obsess.

The coverage **Statement** coverage measures how many statements in the code were executed

Lets see what our coverage looks like!

In [ ]:
!python -m pip install coverage

In [ ]:
!python -m coverage run -m pytest

Well that looks exactly like our usual `pytest` output...

In [ ]:
!python -m coverage report -m

***That's pretty good!***

In case you're wondering what those headings mean:  

**Stmts** The total number of statements in the package.  
**Miss** The number of statements that were not executed during testing.  
**Cover** The percentage of statements that were executed during testing.

But say we don't want to have to go poking around in the files trying to figure out where we missed some opportunities to write tests. Say we rather point and click.

In [ ]:
!python -m coverage html

In [ ]:
from IPython.display import display, HTML
file_path = 'htmlcov/index.html'
with open(file_path, 'r') as file:
    html_content = file.read()
display(HTML(html_content))

**A real example from [FastAPI](https://github.com/tiangolo/fastapi), just check on their impressive 100% code coverage badge to see their report!**

## Formatting / Linting

This might seem aesthetic, but with large, collaborative projects, maintaining conformance to predefined code formatting rules can make a big difference in **readability** and **maintainability**.

Consider two ways that the same Python function definition can be written:

```python
    def my_command(arg1, arg2, arg3, arg4, arg5, arg6, arg7, arg8, arg9, arg10)
```

```python
    def my_command(
        arg1, 
        arg2, 
        arg3, 
        arg4, 
        arg5, 
        arg6, 
        arg7, 
        arg8, 
        arg9, 
        arg10)
```

While functionally equivalent, they are very different in style and mixing the two formats in one codebase would decrease readability.

There are many tools for linting/formatting, such as:
- **Python**: flake8, Ruff
- **C++**: clang-format, cpplint
- **Javascript**: eslint

Let's give ruff a try on our existing Python code:

In [ ]:
!python -m pip install ruff

In [ ]:
!python -m ruff check test/ gradebook/ solutions/

Ruff reports a few code quality problems, and tells us that it can automatically fix them. You're welcome to do that, but let's also see what Ruff thinks of our code formatting:

In [ ]:
!python -m ruff format --check test/ gradebook/ solutions/

And Ruff would be happy to reformat the code for us if we removed the `--check` flag.